In [17]:
!pip install -qU crewai

In [22]:
from crewai import Agent, Task, Crew, LLM
from textwrap import dedent

In [18]:
import os
os.environ["GROQ_API_KEY"] = ""  # Replace with your Groq API key

In [29]:
llm = LLM(model="groq/llama-3.1-70b-versatile")

In [4]:
def get_user_requirements():
    print("\n=== Project Requirements ===")
    project_description = input("Enter the project description: ")
    language = input("Enter the programming language (default is Python): ") or "Python"
    specific_requirements = input("Enter any specific requirements (optional): ")

    # Combine all requirements into a detailed task description
    task_description = f"""
    Create a program with the following specifications:

    Project Description: {project_description}
    Programming Language: {language}
    Additional Requirements: {specific_requirements}

    Please provide:
    1. Well-structured code
    2. Proper documentation
    3. Type hints (if applicable)
    4. Error handling
    5. Examples of usage
    """
    return dedent(task_description)

In [30]:
# Define the Developer Agent
developer = Agent(
    role='Senior Software Developer',
    goal='Write clean, efficient, and well-documented code',
    backstory="""You are an experienced developer with strong skills in
    writing maintainable and efficient code. You take feedback well and are
    committed to continuous improvement.""",
    verbose=True,
    llm=llm
)


In [31]:
# Define the Code Reviewer Agent
reviewer = Agent(
    role='Code Reviewer',
    goal='Ensure code quality and suggest improvements',
    backstory="""You are a meticulous code reviewer with years of experience in
    identifying potential issues and suggesting improvements. You focus on code
    quality, performance, and best practices.""",
    verbose=True,
    llm=llm
)


In [49]:
# Get user requirements
user_task_description = get_user_requirements()


=== Project Requirements ===
Enter the project description: Fetch random cartoon image from XKCD
Enter the programming language (default is Python): 
Enter any specific requirements (optional): Keep it crisp


In [50]:
write_code_task = Task(
    description=user_task_description,
    agent=developer,
    expected_output="The expected output is the code generated by the developer agent based on the user's requirements. Don't include any backtick or meta data, just give me the code"
)

In [51]:
# Task 2: Review the code
review_code_task = Task(
    description="""Review the code provided by the developer. Check for:
    1. Code correctness
    2. Performance optimization opportunities
    3. Documentation quality
    4. Type hint usage
    5. Edge cases handling
    Provide specific recommendations for improvement.""",
    agent=reviewer,
    expected_output="Code review feedback and suggestions for improvement.Don't include any backtick or meta data, just give me the code"
)

In [52]:
# Task 3: Implement review feedback
implement_feedback_task = Task(
    description="""Review the feedback provided and implement the suggested
    improvements. Ensure all points are addressed and document any changes
    made.""",
    agent=developer,
    expected_output="Improved code incorporating the review feedback.Don't include any backtick or meta data, just give me the code"
)

In [53]:
# Create the crew
code_crew = Crew(
    agents=[developer, reviewer],
    tasks=[write_code_task, review_code_task, implement_feedback_task],
    verbose=True
)

In [54]:
# Execute the crew's tasks
print("\nStarting the development and review process...")
result = code_crew.kickoff()


Starting the development and review process...
# Agent: Senior Software Developer
## Task: 
Create a program with the following specifications:

Project Description: Fetch random cartoon image from XKCD
Programming Language: Python
Additional Requirements: Keep it crisp

Please provide:
1. Well-structured code
2. Proper documentation
3. Type hints (if applicable)
4. Error handling
5. Examples of usage



# Agent: Senior Software Developer
## Final Answer: 
import requests
from PIL import Image
from io import BytesIO
import os

def fetch_random_xkcd_image() -> str:
    """
    Fetches a random cartoon image from XKCD and saves it to the current directory.

    Returns:
        str: The filename of the saved image.
    """
    try:
        # Send a GET request to the XKCD API to get a random comic
        response = requests.get('https://c.xkcd.com/random/comic/')
        
        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
   

In [55]:
print(result.raw)

def fetch_random_xkcd_image() -> str:
    """
    Fetches a random cartoon image from XKCD and saves it to the current directory.

    Args:
        None

    Returns:
        str: The filename of the saved image.

    Raises:
        Exception: If an error occurs during the fetching or saving of the image.
        IOError: If an I/O error occurs while saving the image.
        ValueError: If the image URL is invalid or the image cannot be opened.
    """
    try:
        # Send a GET request to the XKCD API to get a random comic
        response = requests.get('https://c.xkcd.com/random/comic/')
        
        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the JSON data from the response
            data = response.json()
            
            # Get the image URL from the JSON data
            image_url = data['img']
            
            # Check if the image URL is valid
            if not image_url:
 